In [45]:
# 기본 설정 및 라이브러리 임포트(데이터 전처리 및 모델 학습에 필요한 여러 라이브러리)
import os            # 운영 체제와 상호작용하기 위한 모듈
import random        # 난수 생성 및 시드 고정을 위한 모듈
import numpy as np   # 수치 연산 및 배열 처리를 위한 라이브러리
import pandas as pd  # 데이터 분석 및 조작을 위한 라이브러리
from ase.io import read  # XYZ 파일을 읽어들이기 위한 모듈 (Atomic Simulation Environment)
import torch         # 딥러닝 모델을 구축하고 학습하기 위한 PyTorch 라이브러리
import torch.nn as nn       # 신경망 관련 모듈
import torch.optim as optim # 최적화 알고리즘 관련 모듈
from torch.utils.data import Dataset, DataLoader, TensorDataset  # 데이터셋 관리 및 로드 모듈
from tqdm.auto import tqdm  # 코드 실행의 진행 상태를 시각적으로 표시하는 모듈


"""
import 모듈
import 모듈 1, 모듈 2
모듈. 변수
모듈. 함수()
모듈. 클래스()

import 모듈 as 이름

from 모듈 import 변수
from 모듈 import 함수
from 모듈 import 클래스
from 모듈 import 변수, 함수, 클래스
from 모듈 import *

from import *     모듈의 모든 변수, 함수, 클래스를 가져올 수 있음.

rom 모듈 import 변수 as 이름
from 모듈 import 함수 as 이름
from 모듈 import 클래스 as 이름
from 모듈 import 변수 as 이름 1, 함수 as 이름 2, 클래스 as 이름 3

import 라이브러리.모듈 
라이브러리.모듈.함수()

"""

# 출력 옵션 설정
np.set_printoptions(threshold=np.inf) # numpy 배열의 모든 요소가 출력되도록 설정

# 시드 고정 함수
def seed_everything(seed):
    random.seed(seed)                   # 파이썬 기본 난수 생성기 시드 고정
    np.random.seed(seed)                # numpy 난수 생성기 시드 고정
    os.environ["PYTHONHASHSEED"] = str(seed)  # 파이썬 해시 시드 고정
    torch.manual_seed(seed)             # PyTorch 난수 생성기 시드 고정

seed_everything(42)  # 시드 값 42로 고정
    


In [46]:
# 데이터 로드 및 기본 탐색 # 데이터를 모델에 맞게 전처리하고 분석하기 전에 데이터를 불러오는 과정

# 데이터 로드
train = read('C:/Users/방우영/Downloads/MLFF_Semiconductor_Simulation/data/Train.xyz', format='extxyz', index=':') # 전체 훈련 데이터 불러오기
test = read('C:/Users/방우영/Downloads/MLFF_Semiconductor_Simulation/data/Test.xyz', format='extxyz', index=':') # 전체 테스트 데이터 불러오기
sample = pd.read_csv('C:/Users/방우영/Downloads/MLFF_Semiconductor_Simulation/data/sample_submission.csv') # 제출 양식 파일 불러오기

# 데이터 개수 출력
print(f"훈련 데이터의 개수: {len(train)}")
print(f"테스트 데이터의 개수: {len(test)}")


훈련 데이터의 개수: 1500
테스트 데이터의 개수: 3000


In [47]:
# 데이터 전처리 단계
# 1. 원자 위치 및 힘 정보 추출

# 초기화
sequence_train, symbols, positions_x, positions_y, positions_z, forces, energies = [], [], [], [], [], [], []

# 훈련 데이터 전처리
for i in range(len(train)):
    mole = train[i]  # 각 분자 구조를 가져옴
    
    atoms = len(mole)  # 해당 분자에 포함된 원자의 수
    sequence_train.append(atoms)  # 각 분자 구조의 원자 수 저장
    
    position = mole.get_positions()  # 원자 위치 정보 (x, y, z 좌표)
    force = mole.get_forces()  # 각 원자에 작용하는 힘
    energy = mole.get_total_energy()  # 전체 분자의 에너지
    energies.append(energy)
    
    for j in range(atoms):  # 각 원자에 대해 반복
        positions_x.append(position[j][0])
        positions_y.append(position[j][1])
        positions_z.append(position[j][2])
        forces.append(force[j])

print(f"positions_x: {len(positions_x)}") # 이 작업 하는 이유 데이터프레임 구조 짜는데 개수가 달라서
print(f"positions_y: {len(positions_y)}")
print(f"positions_z: {len(positions_z)}")
print(f"forces: {len(forces)}")
print(f"energies: {len(energies)}")

expanded_energies = []

for i in range(len(train)):
    mole = train[i]
    atoms = len(mole)  # 해당 분자에 포함된 원자 수
    expanded_energies.extend([energies[i]] * atoms)  # 에너지를 원자 수만큼 반복하여 추가

# 확인
print(len(expanded_energies))  # 144000이어야 함



positions_x: 144000
positions_y: 144000
positions_z: 144000
forces: 144000
energies: 1500
144000


In [48]:
# 2. 데이터프레임 생성 (import pandas as pd)

# DataFrame 생성
train_df = pd.DataFrame({
    'position_x': positions_x, 
    'position_y': positions_y, 
    'position_z': positions_z, 
    'force': forces,
    'energy': expanded_energies  # 확장된 에너지 리스트 사용
})

# 데이터프레임 확인
print(train_df.head())


# 데이터 확인
train_df.head()

   position_x  position_y  position_z                                   force  \
0    2.230816    8.155257    6.391140   [0.08813055, -0.90894865, 1.04011568]   
1    5.820498    5.539081    6.063752   [0.49469689, -0.23481429, 1.14418526]   
2    0.649109    8.043429    9.162340  [-0.47646964, 1.67774442, -1.52065335]   
3    7.276341    7.946647    9.368211    [0.70496183, 1.83900631, 1.37385827]   
4    7.695766    7.129786    3.224149   [-0.09108712, -1.1645404, 1.44755996]   

       energy  
0 -945.262038  
1 -945.262038  
2 -945.262038  
3 -945.262038  
4 -945.262038  


,position_x,position_y,position_z,force,energy
0,2.230816,8.155257,6.391140,"[0.08813055, -0.90894865, 1.04011568]",-945.262038
1,5.820498,5.539081,6.063752,"[0.49469689, -0.23481429, 1.14418526]",-945.262038
2,0.649109,8.043429,9.162340,"[-0.47646964, 1.67774442, -1.52065335]",-945.262038
3,7.276341,7.946647,9.368211,"[0.70496183, 1.83900631, 1.37385827]",-945.262038
4,7.695766,7.129786,3.224149,"[-0.09108712, -1.1645404, 1.44755996]",-945.262038


In [49]:
#테스트 데이터 전처리
# 초기화
sequence_test, positions_x, positions_y, positions_z = [], [], [], []

# 테스트 데이터 전처리
for i in range(len(test)):
    mole = test[i]  # 각 분자 구조를 가져옴

    atoms = len(mole)  # 원자 수
    sequence_test.append(atoms)  # 각 분자 구조의 원자 수 저장

    position = mole.get_positions()  # 원자 위치 정보

    for j in range(atoms):  # 각 원자에 대해 반복
        positions_x.append(position[j][0])
        positions_y.append(position[j][1])
        positions_z.append(position[j][2])

# DataFrame 생성
test_df = pd.DataFrame({
    'position_x': positions_x, 
    'position_y': positions_y, 
    'position_z': positions_z, 
    'force': None  # 테스트 데이터는 레이블이 없으므로 None으로 설정
})

# 데이터 확인
test_df.head()


,position_x,position_y,position_z,force
0,3.434929,4.600871,6.329366,None
1,10.632562,3.276950,6.474922,None
2,3.438480,2.931912,2.621738,None
3,3.042868,6.105789,3.277181,None
4,2.527150,0.367885,0.119567,None


In [50]:
# 4. 데이터셋 준비 및 Pytorch 데이터로더 생성
#from torch.utils.data import Dataset, DataLoader
#import torch

class ForceDataset(Dataset):
    def __init__(self, df, mode='train'):
        self.df = df
        self.mode = mode
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        # 위치 정보 가져오기
        pos_x = self.df.loc[idx, 'position_x']
        pos_y = self.df.loc[idx, 'position_y']
        pos_z = self.df.loc[idx, 'position_z']
        
        # 입력 데이터 생성 (위치 정보)
        inputs = torch.tensor([pos_x, pos_y, pos_z], dtype=torch.float32)
        
        if self.mode != 'test':
            # 훈련 데이터일 경우, 레이블(힘)을 반환
            label = torch.tensor(self.df.loc[idx, 'force'], dtype=torch.float32)
            return inputs, label
        else:
            # 테스트 데이터일 경우, 입력만 반환
            return inputs



In [51]:
#DataLoader 생성
# Dataset 및 DataLoader 생성
train_dataset = ForceDataset(train_df, 'train')
test_dataset = ForceDataset(test_df, 'test')

train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)

# 첫 번째 배치 데이터 확인
for inputs, labels in train_loader:
    print(f"Inputs: {inputs}")
    print(f"Labels: {labels}")
    break  # 첫 번째 배치만 확인

## 전처리 완료


Inputs: tensor([[ 6.9872e-01,  7.2984e+00,  1.0098e+01],
        [ 8.2177e+00,  2.8778e+00,  4.3186e+00],
        [ 5.6105e-01,  6.3018e+00,  9.4110e-01],
        [ 7.5417e+00,  2.5777e+00,  1.2400e+00],
        [ 4.3331e+00,  4.1223e-01,  1.0544e+01],
        [ 1.0675e+01,  2.0284e+00,  6.9086e+00],
        [ 6.0914e+00,  1.8637e-01,  4.1077e+00],
        [ 7.6232e+00,  6.0091e+00,  4.6938e+00],
        [ 6.6030e+00,  5.8465e+00,  3.2379e+00],
        [ 2.6391e+00,  1.3102e+00,  7.6353e+00],
        [ 6.6380e-01,  6.4078e+00,  6.8895e+00],
        [ 1.0519e+01,  3.1243e+00,  1.0124e+01],
        [ 2.7501e+00,  2.8622e+00,  1.1946e+01],
        [ 1.0668e+01,  5.5840e+00,  3.1318e+00],
        [ 7.2493e+00,  6.6002e+00,  7.6052e+00],
        [ 3.2880e+00,  3.4049e-01,  8.5882e+00],
        [ 4.8523e+00,  6.8174e+00,  5.6736e-01],
        [ 9.1047e+00,  6.8759e+00,  3.7374e+00],
        [ 5.2668e+00,  1.4530e+00,  2.0621e+00],
        [ 9.3762e+00,  1.0036e+01,  3.4104e-02],
        [ 6.

In [52]:
# 모델 설계
# import torch.nn as nn

class ForceModel(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(ForceModel, self).__init__()
        
        self.layers = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.BatchNorm1d(hidden_size),
            nn.ReLU(),
            nn.Dropout(0.5),
            
            nn.Linear(hidden_size, 128),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(0.01),
            nn.Dropout(0.5),
            
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.5),
            
            nn.Linear(64, 3)  # 최종 출력은 힘 (x, y, z 3개 값)
        )
    
    def forward(self, x):
        return self.layers(x)


In [53]:
# 손실 함수 및 옵티마이저 설정

# device 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"current device is {device}")

# 모델 초기화
input_size = 3  # 위치 정보 (x, y, z)
hidden_size = 256
model = ForceModel(input_size, hidden_size).to(device)

# 손실 함수와 옵티마이저 정의
criterion = nn.MSELoss()  # 평균 제곱 오차 (Mean Squared Error)
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam 옵티마이저


current device is cpu


In [54]:
# 모델 학습
num_epochs = 3

print("Training Start!")
model.train()  # 모델을 학습 모드로 설정

for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    
    for inputs, labels in tqdm(train_loader):
        optimizer.zero_grad()  # 옵티마이저 초기화
        
        inputs = inputs.to(device)  # 데이터를 GPU로 이동
        labels = labels.to(device)  # 레이블도 GPU로 이동
        
        outputs = model(inputs)  # 모델 예측
        loss = criterion(outputs, labels)  # 손실 계산
        
        loss.backward()  # 역전파 수행
        optimizer.step()  # 가중치 업데이트

print("Training Complete!")

# 에너지 모델 학습

class EnergyModel(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EnergyModel, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 1)  # 에너지를 예측하기 위해 출력 크기는 1
        )
        
        # 에너지 불확실성 추가 예측
        self.uncertainty_layer = nn.Linear(hidden_size, 1)

    def forward(self, x):
        features = self.layers[:-1](x)
        energy = self.layers[-1](features)
        uncertainty = torch.exp(self.uncertainty_layer(features))  # 불확실성은 양수로 변환하기 위해 exp 사용
        return energy, uncertainty
    
# 데이터 준비 (X는 원자 위치 정보, y는 에너지 값)
X_tensor_train = torch.tensor(train_df[['position_x', 'position_y', 'position_z']].values, dtype=torch.float32)
y_tensor_train = torch.tensor(train_df['energy'].values, dtype=torch.float32).view(-1, 1)

# TensorDataset과 DataLoader를 사용하여 데이터를 PyTorch에서 사용 가능하도록 준비
train_dataset = TensorDataset(X_tensor_train, y_tensor_train)
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = EnergyModel(input_size=3, hidden_size=256).to(device)

criterion = nn.MSELoss()  # 에너지 예측의 손실 함수는 MSELoss 사용
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam 옵티마이저 사용

num_epochs = 5

print("Training Start!")
model.train()

for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    
    for inputs, labels in train_loader:
        optimizer.zero_grad()  # 옵티마이저 초기화
        
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        energy_preds, _ = model(inputs)  # 모델 출력에서 에너지 예측값만 사용
        loss = criterion(energy_preds, labels)  # 손실 계산
        
        loss.backward()  # 역전파 수행
        optimizer.step()  # 가중치 업데이트


print("Training Complete!")

model.eval()  # 모델을 평가 모드로 전환

preds = []
predicted_energies = []
predicted_uncertainties = []

model.eval()  # 모델을 평가 모드로 전환

with torch.no_grad():
    for inputs in test_loader:
        inputs = inputs.to(device)
        
        energy_preds, uncertainty_preds = model(inputs)  # 모델 출력이 튜플이므로 분리
        predicted_energies.extend(energy_preds.cpu().numpy())  # 에너지 예측 결과를 리스트에 저장
        predicted_uncertainties.extend(uncertainty_preds.cpu().numpy())  # 불확실성 예측 결과를 리스트에 저장

print("Inference Complete!")


Training Start!
Epoch 1/3


  0%|          | 0/563 [00:00<?, ?it/s]

Epoch 2/3


  0%|          | 0/563 [00:00<?, ?it/s]

Epoch 3/3


  0%|          | 0/563 [00:00<?, ?it/s]

Training Complete!
Training Start!
Epoch 1/5
Epoch 2/5
Epoch 3/5
Epoch 4/5
Epoch 5/5
Training Complete!
Inference Complete!


In [56]:
print("Inference Start!")
model.eval()  # 모델을 평가 모드로 설정

predicted_energies = []
predicted_uncertainties = []
preds = []

with torch.no_grad():  # 예측 시에는 그래디언트 계산이 필요 없으므로 no_grad() 사용
    for inputs in tqdm(test_loader):
        inputs = inputs.to(device)
        
        # 모델의 출력은 튜플이므로, 각 요소를 분리합니다.
        energy_preds, uncertainty_preds = model(inputs)
        
        # 각각의 예측값을 cpu()로 변환 후 numpy로 변환하여 리스트에 추가합니다.
        predicted_energies.extend(energy_preds.cpu().numpy())
        predicted_uncertainties.extend(uncertainty_preds.cpu().numpy())
        preds.extend(energy_preds.cpu().numpy())  # 예시로 에너지 예측값을 preds에 저장

print("Inference Complete!")

print(len(preds))  # preds 리스트의 길이 확인
print(len(sample))  # sample 데이터프레임의 행 수 확인

# 분자별로 힘 데이터를 그룹화
bundled_preds = []
start = 0

for num_atoms in sequence_test:  # sequence_test에는 각 분자별 원자 수가 저장되어 있음
    end = start + num_atoms
    bundled_preds.append(preds[start:end])
    start = end

# 이제 bundled_preds는 3000개의 분자별로 힘을 그룹화한 리스트가 됩니다.

sample['force'] = [str(pred) for pred in bundled_preds]  # 힘 데이터를 문자열로 변환하여 저장


# 분자별로 에너지 예측 값을 평균
average_predicted_energies = []
average_predicted_uncertainties = []

start = 0

for num_atoms in sequence_test:
    end = start + num_atoms
    
    avg_energy = np.mean(predicted_energies[start:end])
    avg_uncertainty = np.mean(predicted_uncertainties[start:end])
    
    average_predicted_energies.append(avg_energy)
    average_predicted_uncertainties.append(avg_uncertainty)
    
    start = end

# 결과를 sample 데이터프레임에 저장
sample['energy'] = average_predicted_energies
sample['energy_uncertainty'] = average_predicted_uncertainties

# 제출 파일 생성
sample.to_csv('submission.csv', index=False)
print("Submission file created: submission.csv")


Inference Start!


  0%|          | 0/1125 [00:00<?, ?it/s]

Inference Complete!
288000
3000
Submission file created: submission.csv
